In [ ]:
#Library imports

#General
import pandas as pd
import numpy as np
import re
import math, time, random, datetime
import time

#Data Exploration
from ydata_profiling import ProfileReport
import uszipcode
from uszipcode import SearchEngine
import sweetviz as sv

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 

#import for interactive plotting
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
%matplotlib inline
sns.set()

In [ ]:
# machine learning
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron,SGDClassifier,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV, learning_curve, cross_val_score
from sklearn.inspection import permutation_importance

from catboost import CatBoostClassifier, Pool, cv

import joblib
import pickle


import warnings 
warnings.filterwarnings('ignore') 

In [ ]:
#reading in the data
df_grade = pd.read_csv("SOECS_clean_grade_data_2.csv")
df_address = pd.read_csv("SOECS_clean_pr_address_2.csv")
df_retention = pd.read_csv("SOECS_clean_retention_data_2.csv")

##### Grade Data Exploration

In [ ]:
#intial view of the first three rows of the data

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows",None)

pd.set_option('display.max_columns', None)
df_grade.head(3)

In [ ]:
#Check to see unique quantities of each feature
df_grade.nunique()

Will remove Camp_Desc in the data cleaning section as it only has 1 value.

In [ ]:
#Checking the various different majors to compare to degrees
value_counts = df_grade['Curr_1_1_Majr_Desc'].value_counts()
print(value_counts.head(10))
print(value_counts.tail(10))

In [ ]:
#Checking the different values for year term 
x = df_grade.Year_Term.unique()
x=np.sort(x)
x

In [ ]:
#Comparing to degrees
value_counts = df_grade['Curr_1_Degc_Desc'].value_counts()
print(value_counts)

In [ ]:
#checking the type of data and for null values
df_grade.info()

There are null values in the last few columns that need to be cleaned

In [ ]:
#Checking the percentage of NaN rows in data
num_rows_with_nan = df_grade.isnull().any(axis=1).sum()
print("percent of rows with Nan:",round(num_rows_with_nan/df_grade.shape[0]*100,3),"%")

About 1% of the rows have Nan values, so they will all be removed in the data cleaning section

In [ ]:
#Finding an example where theres more than 1 Crse_Name for corresponding Crse_Title.

# Count the number of unique values of crse_name for each crse_title
crse_counts = df_grade.groupby('Crse_Title')['Crse_Name'].nunique()

# Find the first crse_title with more than one or zero unique crse_name
first_mismatch = crse_counts[(crse_counts != 1)].index[0]

print(f"The first crse_title with either more than 1 or 0 unique crse_name is {first_mismatch}")


In [ ]:
#Checking to see
filtered_column = df_grade[df_grade['Crse_Title'] == 'Academic Skills for Success']['Crse_Name']

# get the unique values in the filtered column
unique_values = filtered_column.unique()

# print the unique values
print(unique_values)

There can be more than 1 Subj_Code per Crse_Title, so drop Crse_Title durring cleaning for being redundant.

In [ ]:
#There was more Grde_Code than the standard grades, so regular expression was used to find all the unexpected Grde_Codes

# Get all unique values in the Grde_Code column
all_grades = df_grade['Grde_Code'].unique()

# Define a regular expression pattern to match non-letter grades
pattern = r'[^A-Fa-f+-]'

# Filter the list of grades based on the pattern
non_letter_grades = [grade for grade in all_grades if re.search(pattern, grade)]

print(f"All non-letter grades in the Grde_Code column are: {non_letter_grades}")


In [ ]:
#Num of Students durring each semester

#removing the unwanted Year_Term values
x2=x[6:24]
df_grade = df_grade[df_grade['Year_Term'].isin(x2)]


# Grouping by Year_Term and counting unique PIDM
df_count = df_grade.groupby('Year_Term')['PIDM'].nunique().reset_index()

# Creating the count plot
plt.figure(figsize=(12, 6)) # Adjust the figure size if needed
ax = sns.barplot(x='Year_Term', y='PIDM', data=df_count)
plt.title('Total Unique PIDM count per Year-Term')
plt.xlabel('Year-Term')
plt.ylabel('Unique PIDM Count')

# Adding count values at the top of each bar
for i, row in df_count.iterrows():
    ax.text(row.name, row.PIDM + 0.5, row.PIDM, ha='center', color='black')

plt.show()


In [ ]:
# Define a dictionary to store the percentage reductions
reductions = {}

# Loop through the Year-Term values and calculate the percentage reduction
for i in range(len(x2)):
    if x2[i].endswith('S'):
        year = int(x2[i][:-1])
        count_s = df_count.loc[df_count['Year_Term'] == x2[i], 'PIDM'].values[0]
        if f'{year}F' in x2:
            count_f = df_count.loc[df_count['Year_Term'] == f'{year}F', 'PIDM'].values[0]
            percentage_reduction = ((count_s - count_f) / count_s) * 100
            reductions[f'{year}F to {year}S'] = round(percentage_reduction, 2)

# Sort the Year-Term pairs alphabetically, ex2cept for '10F to 10S'
reductions = dict(sorted(reductions.items(), key=lambda x2: x2[0] != '10F to 10S'))


# Print the percentage reductions
for years, reduction in reductions.items():
    print(f'{years}: {reduction}% reduction')


##### Grade Data Cleaning

In [ ]:
#per Dr. Gupta meeting with Dr. Hetrick: remove Computer Information Systems 
df_grade=df_grade[df_grade['Curr_1_1_Majr_Desc']!='Computer Information Systems']

In [ ]:
#Removing the # values in grade cells
df_grade['Grde_Code'] = df_grade['Grde_Code'].str.replace('#', '')

In [ ]:
#droping NaN values
df_grade = df_grade.dropna(how='any')

In [ ]:
#Confirming all NaN values are dropped
df_grade.info()

In [ ]:
#Reducing the amount of features by combining the first and last name columns to one column

# Combine the first and last name columns
df_grade['Advr_1_Full_Name'] = df_grade['Advr_1_First_Name'].str.cat(df_grade['Advr_1_Last_Name'], sep=' ')

# Drop the first and last name columns
df_grade=df_grade.drop(['Advr_1_First_Name', 'Advr_1_Last_Name'], axis=1)


In [ ]:
#Dropping Redundant Features in Grade Data

#only 1 value in the column
df_grade=df_grade.drop(['Camp_Desc'], axis=1)

#Redundant with Curr_1_1_Majr_Desc
df_grade=df_grade.drop(['Curr_1_Degc_Desc'], axis=1)

#Redundant with Term_Desc
df_grade=df_grade.drop(['Term_Code'], axis=1)

#Redundant with Study_Year 
df_grade=df_grade.drop(['Year_Term'], axis=1)

#Redundant with Crse_Name + Instructor Name
df_grade=df_grade.drop(['CRN'], axis=1)

#Redundant with Crse_Name
df_grade=df_grade.drop(['Subj_Code'], axis=1)
df_grade=df_grade.drop(['Crse_Numb'], axis=1)
df_grade=df_grade.drop(['Crse_Title'], axis=1)

#Different Teachers can have same section numbers probably will confuse model
df_grade=df_grade.drop(['Sect_Numb'], axis=1)

#Redundant with already existing GPA information
df_grade=df_grade.drop(['Credit_Hr'], axis=1)
df_grade=df_grade.drop(['Grde_Code'], axis=1)

In [ ]:
#Information regarding major 
value_counts = df_grade['Curr_1_1_Majr_Desc'].value_counts()
print(value_counts.head(5))
print(value_counts.tail(5))

In [ ]:
#droping duplicate rows
df_grade.drop_duplicates(inplace=True)

In [ ]:
#Final look at grade data after intial data cleaning
df_grade.reset_index(drop=True, inplace=True)
print(df_grade.shape)
df_grade.head(3)

In [ ]:
#per meeting with Dr. Hetrick, find top 5 most common classes for students by year
df_grade.Study_Year.value_counts()

Will opt to only do this for the first five years.

In [ ]:
# select data for first 4 study years
df_grade_1234 = df_grade[df_grade['Study_Year'].isin([1, 2, 3, 4, 5])]

# get the top 5 most common Crse_Name for each year
top_5 = df_grade_1234.groupby(['Study_Year', 'Crse_Name'])['Crse_Name'].count().reset_index(name='count').sort_values(['Study_Year', 'count'], ascending=[True, False]).groupby('Study_Year').head(5)

#Visual for the classes
top_5

In [ ]:
#Creating a new data frame to fill the data in
top_5_grades = pd.DataFrame({'PIDM': df_grade['PIDM'].unique()})
#Creating columns for each of the year and class combination
for i, row in top_5.iterrows():
    col_name = 'Study_Year ' + str(row['Study_Year']) + ' Crse_Name ' + row['Crse_Name']
    top_5_grades[col_name] = np.nan

In [ ]:
#Populating the data
#side note takes ~5mins to run
for i, row in top_5_grades.iterrows():
    pidm = row['PIDM']
    # Loop through columns of top_5_grades
    for col in top_5_grades.columns[1:]:
        study_year = col.split(' ')[1]
        crse_name = col.split(' ')[3]
        # Find matching row in df_grade
        match = df_grade[(df_grade['PIDM'] == pidm) & (df_grade['Study_Year'] == int(study_year)) & (df_grade['Crse_Name'] == crse_name)]
        if len(match) > 0:
            top_5_grades.loc[i, col] = match.iloc[0]['Grde_Code_Qlty_Pnts']
        else:
            top_5_grades.loc[i, col] = -1

In [ ]:
#viewing the new columns that will be added to the final data set
top_5_grades.head(3)

#### Address Data Exploration

In [ ]:
#intial view of the data
df_address.head(3)

In [ ]:
#Checking the amount of unique values in each feature of the data
df_address.nunique()

In [ ]:
#Checking the amount of NaN and type of data 
df_address.info()

Since PR_NATN_CODE and PR_NATN_DESC have so many nulls, so durring the Data Cleaning the data will be filled for PR_NATN_DESC. PR_NATN_CODE does not tell me anything meanigful, so it will be removed later.

#### Address Data Cleaning

In [ ]:
#Creating Zip code with only first 5 digits

# Define a regular expression pattern to match the first five digits
pattern = r'^(\d{5})'

# Apply the pattern to the PR_ZIP column using the str.extract() method
df_address['ZIP'] = df_address['PR_ZIP'].str.extract(pattern)

df_address.head(3)

In [ ]:
#Droping data

#Redundant with PR_STAT_CODE
df_address=df_address.drop('PR_STAT_DESC', axis=1)

#Redundant with ZIP
df_address=df_address.drop(['PR_ZIP'], axis=1)


#Redundant with PR_NATN_DESC
df_address=df_address.drop(['PR_NATN_CODE'], axis=1)

df_address.head(3)

In [ ]:
#saving a copy to add back to, allowing for median salary to be given to the values that do not have a ZIP value
df_address2=df_address

#Removing all NaN in ZIP
df_address = df_address.dropna(subset=['ZIP'])
df_address.info()

#Since I want to find the average household income, the data with no ZIP is removed
df_address = df_address.reset_index(drop=True)

df_address.head(3)

In [ ]:
#Since there was some U.S. states that were not didn't have a PR_NATN_DESC, I filled all them as UNITED STATES
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
df_address_us_states = df_address[df_address['PR_STAT_CODE'].isin(us_states)]

df_address.loc[df_address_us_states.index, 'PR_NATN_DESC'] = 'UNITED STATES'

df_address.info()

In [ ]:
#cleaning PR_NATN_DESC nulls
nan_rows = df_address[df_address['PR_NATN_DESC'].isna()]
nan_rows

In [ ]:
#I manually looked the locations up
df_address.loc[690, 'PR_NATN_DESC'] = 'UNITED STATES'
df_address.loc[1461, 'PR_NATN_DESC'] = 'UNITED STATES'
df_address.loc[1601, 'PR_NATN_DESC'] = 'UNITED STATES'

In [ ]:
#Checking again on NaNs
df_address.info()

In [ ]:
#checking on the rows with no PR_STAT_CODE
nan_rows = df_address[df_address['PR_STAT_CODE'].isna()]
nan_rows.head(3)

In [ ]:
#Checking on the quantity the NaNs are from foreign countries
missing_states = df_address[df_address['PR_STAT_CODE'].isna()]
natn_desc_counts = missing_states['PR_NATN_DESC'].value_counts()
print(natn_desc_counts)
sum(natn_desc_counts)

In [ ]:
#Put the words No State for the ones that have no stat
df_address[df_address['PR_STAT_CODE'].isna()] = df_address[df_address['PR_STAT_CODE'].isna()].fillna("No State")

df_address.info()

In [ ]:
#US zip code only has data on US, so U.S. will be subsetted
df_address_US=df_address[df_address['PR_NATN_DESC']=='UNITED STATES']

In [ ]:
#Creating the median household income through zip code information

# Create a SearchEngine instance
search = SearchEngine()


# Define a function to get the median household income for a ZIP code
def get_median_income(zipcode):
    try:
        return search.by_zipcode(zipcode).median_household_income
    except:
        return 'N/A'

# Runing the function
df_address_US.loc[:, 'median_household_income'] = df_address_US['ZIP'].apply(get_median_income)

In [ ]:
#Checking to see if the median household income had any issues
df_address_US.info()

There are unaccounted for null values, I will drop these values and fill them up with the median income of the data.

In [ ]:
#drop null values
df_address_US=df_address_US.dropna()
#making sure they are dropped
df_address_US.info()

In [ ]:
#creating median filler for all the nulls
median_income = df_address_US['median_household_income'].median()

In [ ]:
#merging the data frames
selected_cols = df_address_US[['PIDM', 'median_household_income']]
merged_df = df_address2.merge(selected_cols, on='PIDM', how='left')
merged_df
merged_df.info()

In [ ]:
# Replace null values in the 'median_household_income' column with 'median_income'
merged_df['median_household_income'].fillna(median_income, inplace=True)
df_address=merged_df
df_address.info()

#### Retention Data Exploration

In [ ]:
#Viewing the first 3 rows of the data
df_retention.head(3)

In [ ]:
#Checking unique values per feature
df_retention.nunique()

The features, DISCIPLINE, college, retrn_cohort and grad_cohort only have 1 value, so they will be removed durring the data cleaning

In [ ]:
#checking the various majors
value_counts = df_retention['major'].value_counts()
print(value_counts)

Majors are not the same as the degrees majors in Grades data set, there are also alot less majors in this data set

In [ ]:
#Checking data types and null values counts
df_retention.info()

I see that there will be NaN that will be cleaned in the data cleaning section.

In [ ]:
#Checking the various regions
value_counts = df_retention['region'].value_counts()
value_counts

Will be removing International and Missing because neither of them will be able to be searched in the U.S. zipcode library 

#### Retention Data Cleaning

In [ ]:
#drop columns

#only 1 value columns
df_retention = df_retention.drop(['DISCIPLINE', 'college', 'retrn_cohort', 'grad_cohort'], axis=1)

#Redundant with  HSGPA_cat
df_retention = df_retention.drop(['HSGPA'], axis=1)

#Redundant with major
df_retention = df_retention.drop(['degc_code'], axis=1)

#viewing data again
df_retention.info()

In [ ]:
#creating a column to see if someone graduated or didn't 1:Graduated 0:Didn't Graduate
df_retention['graduated'] = df_retention[['grad_3yr', 'grad_4yr', 'grad_5yr', 'grad_6yr']].any(axis=1).astype(int)
df_retention.head(10)

In [ ]:
#2015 and before do not have NaN problem 
#2016 and after has NaN problem due to data only reaching 2020, don't know grad_6yr and before NaN
#ex: 2016: grad_6yr=NaN, 
#    2017: retrn_6yr=Nan grad_5yr&grad_6yr=NaN, 
#    2018: retrn_5yr&retrn_6yr=Nan grad_4yr&grad_5yr&grad_6yr=NaN, etc

#Ex: of 2016
df_retention[df_retention['term_code_key']==201681].head(3)

In [ ]:
#Ex: of 2020
df_retention[df_retention['term_code_key']==202081].head(3)

In [ ]:
#Solution for now subset out the 2016 and after data
df_retention.info()

In [ ]:
#Checking on the years
years=df_retention['term_code_key'].unique()
years

In [ ]:
### PG added
#x = df_retention['term_code_key'].astype(str).str[:4]
df_retention = df_retention[df_retention['term_code_key'].astype(str).str[:4]<'2016']
df_retention.info()

In [ ]:
#subsetting data for 2015 and before
years_2015_and_before=years[:11]
df_retention2015 = df_retention[df_retention['term_code_key'].isin(years_2015_and_before)]
df_retention2015.info()

Large subset of data is lost due to removal of students from years 2016 and onward.

In [ ]:
#checking on how balanced the data is
df_retention2015.graduated.value_counts()

In [ ]:
#EDA of cleanned data
#profile = ProfileReport(df_retention2015, title="Report")
#profile

#### Joining the Data Frames

In [ ]:
#creating columns Crse_Name,Inst_GPA, Curr_1_1_Majr_Desc
df_grade_groupby=df_grade.groupby("PIDM",as_index=False, sort = False).aggregate({'Crse_Name':'size', "Inst_GPA":'mean','Curr_1_1_Majr_Desc':'nunique'})

df_grade_groupby=df_grade_groupby.rename(columns={'Crse_Name': "Num_of_Crses","Inst_GPA": "Mean_GPA",'Curr_1_1_Majr_Desc': "Num_of_Unique_Majors"})
df_retention_plus_df_grade = pd.merge(df_grade_groupby,df_retention2015, on='PIDM', how = 'inner')

In [ ]:
#joining the USA address data to the rest of the data
df_address_income=df_address[['PIDM', 'median_household_income']]
df_combined = pd.merge(df_address_income,df_retention_plus_df_grade, on='PIDM', how = 'inner')

df_address_income_AE=df_address[['PIDM', 'median_household_income', 'ZIP']]
df_combined_AE = pd.merge(df_address_income_AE,df_retention_plus_df_grade, on='PIDM', how = 'inner')

In [ ]:
#joining the combined data with the top 5 classes in each semester
df_combined = pd.merge(df_combined,top_5_grades, on='PIDM', how = 'inner')

In [ ]:
#Viewing combined data
df_combined.head(3)
#drop term_code_key

In [ ]:
#checking on the shape of the data set
df_combined.shape

In [ ]:
#Checking again on the data
df_combined.info()

In [ ]:
#making all the data types consistent for retrn years
df_combined['retrn_2yr'] = df_combined['retrn_2yr'].astype('float64')

#### EDA of Combined Data Frame

In [ ]:
df_cat = df_combined.select_dtypes('object')
df_num = df_combined.select_dtypes(exclude = [object])

In [ ]:
df_num.shape

In [ ]:
df_num.columns

In [ ]:
df_num_no_crse_name = df_num[['median_household_income', 'Num_of_Crses', 'Mean_GPA',
       'Num_of_Unique_Majors', 'term_code_key', 'retrn_2yr', 'retrn_3yr',
       'retrn_4yr', 'retrn_5yr', 'retrn_6yr','graduated']]

In [ ]:
retrn_data = df_combined[['retrn_2yr','retrn_3yr', 'retrn_4yr', 'retrn_5yr', 'retrn_6yr']]


In [ ]:
for i in retrn_data.columns:
    plt.figure(figsize=(10,4))
    sns.countplot(retrn_data[i], palette = 'hls')
    plt.title("Distribution By Return Year")
    #sns.countplot(df_retrn_data[i], data = , palette = 'hls')
    plt.show()

Each year the number of students who return decreases, which makes sense because they are graudating after 4 years usually.

#### Slide 4

In [ ]:
#ax = sns.countplot(x='User', data=df)

ax = sns.countplot(x = df_combined['graduated'])
ax.bar_label(ax.containers[0])
plt.title("Graduated College vs Did Not Graduate College Count", fontsize = 20);

Almost 70% of students graduate from college.

In [ ]:
df_combined.columns

In [ ]:
df_combined_short = df_combined[['median_household_income', 'Num_of_Crses', 'Mean_GPA',
       'Num_of_Unique_Majors', 'term_code_key', 'HSGPA_cat', 'major',
       'ethnicity', 'gender', 'region', 'retrn_2yr', 'retrn_3yr', 'retrn_4yr',
       'retrn_5yr', 'retrn_6yr', 'grad_3yr', 'grad_4yr', 'grad_5yr',
       'grad_6yr', 'graduated']]

df_combined_short_Wo_Grad = df_combined[['median_household_income', 'Num_of_Crses', 'Mean_GPA',
       'Num_of_Unique_Majors', 'term_code_key', 'HSGPA_cat', 'major',
       'ethnicity', 'gender', 'region', 'retrn_2yr', 'retrn_3yr', 'retrn_4yr',
       'retrn_5yr', 'retrn_6yr', 'graduated']]

In [ ]:
df_PIDM = pd.read_csv("PDIM_Combined_df.csv") 

In [ ]:
# Code to Subset all Graduated Students
df_combined2 = df_combined

all_graduated_students_df = df_combined2[(df_combined2["graduated"] == 1)]

In [ ]:
# Code to Subset all Non-Graduated Students

all_non_grad_df = df_combined2[(df_combined2["graduated"] == 0)]

In [ ]:
#### Slide 13

In [ ]:
#Stacked bar chart to compare genders and grad/non grad counts

females_df = df_combined2[(df_combined2["gender"] == "F")]
males_df = df_combined2[(df_combined2["gender"] == "M")]


male_graduates = males_df.loc[males_df['graduated'] == 1, 'gender'].count()
male_non_graduates = males_df.loc[males_df['graduated'] == 0, 'gender'].count()
female_graduates = females_df.loc[females_df['graduated'] == 1, 'gender'].count()
female_non_graduates = females_df.loc[females_df['graduated'] == 0, 'gender'].count()

# Create the stacked bar chart
fig, ax = plt.subplots()

# Set the x-axis tick locations and labels
x_ticks = [0, 1]
x_labels = ['Non Graduates', 'Graduates']


# # Create the bars for males and females
# male_bars = ax.bar(x_ticks, [male_non_graduates, male_graduates], label='Males', color='darkblue') # change the value of color in order to change the color of males legend
# female_bars = ax.bar(x_ticks, [female_non_graduates, female_graduates], bottom=[male_non_graduates, male_graduates],
#        label='Females', color='yellow') # change the value of color in order to change the color of females legend

#Create the bars for males and females
female_bars = ax.bar(x_ticks, [female_non_graduates, female_graduates], label='Females', color='#C0C0C0') # change the value of color in order to change the color of males legend
male_bars = ax.bar(x_ticks, [male_non_graduates, male_graduates], bottom=[female_non_graduates, female_graduates],
       label='Males', color='#233C9B') # change the value of color in order to change the color of females legend

# Set the x-axis label, y-axis label, and chart title
ax.set_xlabel('Graduation')
ax.set_ylabel('Count')
ax.set_title('Graduation Counts by Gender')

# Set the x-axis tick locations and labels
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_labels)

# Add the legend
ax.legend()

for i, rect in enumerate(ax.patches):
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()

    # The height of the bar is the count value and can used as the label
    label_text = f'{height:.0f}'

    label_x = x + width / 2
    label_y = y + height / 2

    # don't include label if it's equivalently 0
    if height > 0.001:
        ax.text(label_x, label_y, label_text, ha='center', va='center', color='white', fontsize=12, fontweight='bold')
        
# Show the plot
plt.show()

#### Slide 12

In [ ]:
# Finding count of student ethnicities
# Pie Chart Created in Excel
# Note: Gray pie chart was created using data from US Census Bureau Website (Referenced in Slides)
# U.S. Census Bureau QuickFacts: Stockton city, California. (n.d.). Www.census.gov. https://www.census.gov/quickfacts/stocktoncitycalifornia 

print(df_combined2["ethnicity"].value_counts())

In [ ]:
#### Slides 17, 18, 19

In [ ]:
# Visualization Slide 17

# BiDirectional Chart that Shows Ethnicites of Grads and Nons (ALL STUDENTS)

grad_data1 = df_combined2[df_combined2['graduated'] == 1]
non_grad_data1 = df_combined2[df_combined2['graduated'] == 0]


grad_counts = list(grad_data1.groupby('ethnicity')['graduated'].count())
non_grad_counts = list(non_grad_data1.groupby('ethnicity')['graduated'].count())

non_grad_counts = [x*-1 for x in non_grad_counts]

ethnicities = list(df_combined2['ethnicity'].unique())

# Create a figure and axis object
fig, ax = plt.subplots()

width = 0.6

ax.barh(ethnicities, grad_counts, height=width)
ax.barh(ethnicities, non_grad_counts, height=width)

# Set the position of the center axis and hide the spines
ax.axvline(x=0, color='black', linewidth=0.5)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Add labels and title
ax.set_xlabel('Number of Individuals')
ax.set_ylabel('Ethnicity')
ax.set_title('Ethnicities of All College Graduates vs. Non-Graduates')

plt.legend(['','Graduate','Non-Graduate'])

plt.show()

In [ ]:
#Slide 17


all_ethnicities_pie_chart = df_combined2

print(all_ethnicities_pie_chart['ethnicity'].value_counts())

In [ ]:
# Visualization Slide 18

# BiDirectional Chart that Shows Ethnicites of Grads and Nons (MALES)

df2_bi_charts =df_combined2


male_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
male_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
grad_data = df2_bi_charts[df2_bi_charts['graduated'] == 1]
non_grad_data = df2_bi_charts[df2_bi_charts['graduated'] == 0]


grad_counts = list(grad_data.groupby('ethnicity')['graduated'].count())
non_grad_counts = list(non_grad_data.groupby('ethnicity')['graduated'].count())

non_grad_counts = [x*-1 for x in non_grad_counts]

ethnicities = list(df_PIDM['ethnicity'].unique())
ethnicities.sort()

# Create a figure and axis object
fig, ax = plt.subplots()

width = 0.5

ax.barh(ethnicities, grad_counts, height=width)
ax.barh(ethnicities, non_grad_counts, height=width)

# Set the position of the center axis and hide the spines
ax.axvline(x=0, color='black', linewidth=0.5)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Add labels and title
ax.set_xlabel('Student Count')
ax.set_ylabel('Ethnicity')
ax.set_title('Ethnicities of Male College Graduates vs. Non-Graduates')

plt.legend(['','Graduate','Non-Graduate'])

plt.show()

In [ ]:
# Slide 18

# What are the ethnicities of students who did graduated?

all_grad_ethnicity_percentage = all_graduated_students_df['ethnicity'].value_counts()
all_grad_ethnicity_percentage

In [ ]:
# Visualization Slide 18

# BiDirectional Chart that Shows Ethnicites of Grads and Nons (MALES)

df2_bi_charts =df_combined2


male_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
male_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
grad_data = df2_bi_charts[df2_bi_charts['graduated'] == 1]
non_grad_data = df2_bi_charts[df2_bi_charts['graduated'] == 0]


grad_counts = list(grad_data.groupby('ethnicity')['graduated'].count())
non_grad_counts = list(non_grad_data.groupby('ethnicity')['graduated'].count())

non_grad_counts = [x*-1 for x in non_grad_counts]

ethnicities = list(df_PIDM['ethnicity'].unique())
ethnicities.sort()

# Create a figure and axis object
fig, ax = plt.subplots()

width = 0.5

ax.barh(ethnicities, grad_counts, height=width)
ax.barh(ethnicities, non_grad_counts, height=width)

# Set the position of the center axis and hide the spines
ax.axvline(x=0, color='black', linewidth=0.5)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Add labels and title
ax.set_xlabel('Student Count')
ax.set_ylabel('Ethnicity')
ax.set_title('Ethnicities of Male College Graduates vs. Non-Graduates')

plt.legend(['','Graduate','Non-Graduate'])

plt.show()

In [ ]:
# Slide 19

# What are the ethnicities of students who did NOT graduate?

all_nongrad_ethnicity_percentage = all_non_grad_df['ethnicity'].value_counts()
all_nongrad_ethnicity_percentage

In [ ]:
# Visualization Slide 19

# BiDirectional Chart that Shows Ethnicites of Grads and Nons (FEMALES)

df2_bi_charts =df_combined2


female_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
female_df = df2_bi_charts[df2_bi_charts['gender'] == 1]
fgrad_data = df2_bi_charts[df2_bi_charts['graduated'] == 1]
fnon_grad_data = df2_bi_charts[df2_bi_charts['graduated'] == 0]


fgrad_counts = list(fgrad_data.groupby('ethnicity')['graduated'].count())
fnon_grad_counts = list(fnon_grad_data.groupby('ethnicity')['graduated'].count())

fnon_grad_counts = [x*-1 for x in fnon_grad_counts]

ethnicities = list(df_PIDM['ethnicity'].unique())
ethnicities.sort()

# Create a figure and axis object
fig, ax = plt.subplots()

width = 0.5

ax.barh(ethnicities, fgrad_counts, height=width)
ax.barh(ethnicities, fnon_grad_counts, height=width)

# Set the position of the center axis and hide the spines
ax.axvline(x=0, color='black', linewidth=0.5)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Add labels and title
ax.set_xlabel('Student Count')
ax.set_ylabel('Ethnicity')
ax.set_title('Ethnicities of Male College Graduates vs. Non-Graduates')

plt.legend(['','Graduate','Non-Graduate'])

plt.show()

In [ ]:
####  Slides 15 & 16

In [ ]:
# Slide 15
# Data for green pie chart (Excel)
# Finding majors of non grad males

male_majors = df_combined2[df_combined2['gender'] == 1] 
male_majors_NG = male_majors[male_majors['major'] == 0] 

print(male_majors_NG['major'].value_counts())

In [ ]:
# Slide 15
# Data for gold pie chart (Excel)
# Finding majors of non grad males

male_majors = df_combined2[df_combined2['gender'] == 1] 
male_majors_G = male_majors[male_majors['major'] == 1] 

print(male_majors_G['major'].value_counts())

In [ ]:
# Slide 16
# Data for blue pie chart (Excel)
# Finding majors of non grad female 

female_majors = df_combined2[df_combined2['gender'] == 0] 
female_majors_NG = female_majors[female_majors['major'] == 0] 

print(female_majors_NG['major'].value_counts())

In [ ]:
# Slide 16
# Data for orange pie chart (Excel)
# Finding majors of non grad female 

female_majors = df_combined2[df_combined2['gender'] == 0] 
female_majors_G = female_majors[female_majors['major'] == 1] 

print(female_majors_G['major'].value_counts())

In [ ]:
# Finding majors of all students who did NOT graduate

majors_all_graduated_students_df = df_combined2
print(all_non_grad_df['major'].value_counts())

In [ ]:
# Finding majors of all students who did graduate

all_graduated_students_df = df_combined2
print(all_graduated_students_df['major'].value_counts())

In [ ]:
#### Slide 20

In [ ]:
# International Students Data

int_students2 = df_PIDM[df_PIDM['ethnicity'] == '1. International']
int_students2 = int_students2[int_students2['graduated'] == 0]
int_students2.shape

In [ ]:
int_students2.head()

In [ ]:
print(int_students2['region'].value_counts())

In [ ]:
print(int_students2['Study_Year 1 Crse_Name PACS1'].value_counts())

In [ ]:
print(int_students2['Mean_GPA'].value_counts())

In [ ]:
print(int_students2['Num_of_Crses'].value_counts())

In [ ]:
print(int_students2['major'].value_counts())

#### Slide 28 Starts

In [ ]:
#The default cmap is sns.cm.rocket. To reverse it set cmap to sns.cm.rocket_r
#Creates a correlation Heatmap of our variables

#Bigger Figure Size
plt.figure(figsize=(15, 15))

cmap = sns.cm.rocket_r
heatmap = sns.heatmap(df_combined.corr(), cmap = cmap)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

The correlation coefficients with the variable "graduated." Based on the coefficients provided, it appears that the variables "retrn_2yr," "retrn_5yr," "grad_4yr," have positive correlations with "graduated." This suggests that as these variables increase, there is a tendency for the "graduated" variable to increase as well. On the other hand, the variables "PIDM," has a negative correlation with "graduated," which indicates that as this variable increases, there is a tendency for the "graduated" variable to decrease.


In [ ]:
#Find the correlation that the "Mean_GPA" column has with the rest.

In [ ]:
df_combined.corrwith(df_combined['Mean_GPA']).round(2)

Mean_GPA and the columns that signify retetion have a positive correlation.
Mean_GPA and graduated have a correlation of 0.63, Mean_GPA and retrn_2yr have a correlation of 0.44, Mean_GPA and retrn_3yr have a correlation of 0.55, Mean_GPA and retrn_4yr have a correlation of 0.56

#### Slide 28 Ends

In [ ]:
#Creates a countplot of graduated vs not graduated by gender
ax = sns.countplot(x = 'graduated',hue = 'gender', data = df_combined)
plt.title("Graduated From College vs Not Graduated From College by Gender", fontsize = 20)
plt.legend(['Male', 'Female']);
for container in ax.containers:
    ax.bar_label(container)

Approximately 76% of females graduated from college compared to approximately 68% of males.

In [ ]:
#Visualizes each major and counts how many graduated vs did not
plt.figure(figsize=(16, 8)) 
ax = sns.countplot(x = 'graduated',hue = 'major', data = df_combined)
plt.title("College Graduation by Major", fontsize = 20);
for container in ax.containers:
    ax.bar_label(container)

Mechanical Engineering had the highest amount of students who did not graduate from college, then Civil Engineering students.

#### Slide 26 Starts

In [ ]:
#Visualizes how many people graduated and did not by region
plt.figure(figsize=(12, 8)) 
sns.set_palette("pastel")
ax = sns.countplot(x = 'region',hue = 'graduated', data = df_combined)
#plt.title("College Graduation by Region", fontsize = 20)

plt.legend(['Did Not Graduate', 'Graduated']);
for container in ax.containers:
    ax.bar_label(container)

Highest population of students comes from Northern CA or Local Market. Approximately 73% of Local Market students graduated from college and 74% of Northern CA graduated from college. International had more students who did not grsduate from college.

In [ ]:
#New Dataframes
#Creates a data frame of just male students.
df_male = df_combined_AE.loc[df_combined_AE['gender'] == "M"]

#Creates a data frame of just female students.
df_female = df_combined_AE.loc[df_combined_AE['gender'] == "F"]

#Creates a data frame of students whose High School GPA was less than 3.10.
df_HSGPA_less310 = df_combined_AE.loc[df_combined_AE['HSGPA_cat'] == "Less 3.10"]

#Creates a data frame of students whose High School GPA was between 3.10 and 3.38.
df_HSGPA_310_to338 = df_combined_AE.loc[df_combined_AE['HSGPA_cat'] == "3.10-3.38"]


In [ ]:
#New Dataframes by region
#Creates a data frame of students that live in the Western region.
df_Western = df_combined_AE.loc[df_combined_AE['region'] == "Western US"]

#Creates a data frame of students that live in the Eastern region.
df_Eastern = df_combined_AE.loc[df_combined_AE['region'] == "Eastern US"]

#Creates a data frame of students that live in the Local Market region.
df_LocalMarket = df_combined_AE.loc[df_combined_AE['region'] == "Local Market"]

#Creates a data frame of students that live in in the Northern California region.
df_NorthernCA = df_combined_AE.loc[df_combined_AE['region'] == "Northern CA"]

#Creates a data frame of students that live in the Southern California region.
df_SouthernCA = df_combined_AE.loc[df_combined_AE['region'] == "Southern CA"]

#Creates a data frame of students that live in in the International region.
df_International = df_combined_AE.loc[df_combined_AE['region'] == "International"]

In [ ]:
#df_Western
#Finds male and femal counts of students that live in the Western region.
df_Western['gender'].value_counts()

A large majority of the Western US region students are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the Western region.
df_Western['ethnicity'].value_counts()

A large majority of students that live in the Western US region are White or Asian.

In [ ]:
#Finds the various zip code counts of students that live in the Western region.
df_Western['ZIP'].value_counts().nlargest(5)

Students in the Western US region come from the zip codes of 96822 and 96734 the most.

In [ ]:
#Finds the count of the students HS GPA category that live in the Western region.
df_Western['HSGPA_cat'].value_counts()

About 50% of the students in the Western US region had HS GPA of 3.60 or higher.

In [ ]:
#df_Eastern
#Finds male and female counts of students that live in the Eastern region.
df_Eastern['gender'].value_counts()

A large majority of students that live in the Eastern US region are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the Eastern region.
df_Eastern['ethnicity'].value_counts()

A large majority of students that live in the Eastern US region are White.

In [ ]:
#Finds the count of the students HS GPA category that live in the Eastern region.
df_Eastern['HSGPA_cat'].value_counts()

About 40% of the students from the Eastern US region had HS GPA of 3.60 or higher

In [ ]:
#Finds male and female counts of students that live in the Local Market region.
df_LocalMarket['gender'].value_counts()

A large majority of students that live in the Local Market region are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the Local Market region.
df_LocalMarket['ethnicity'].value_counts()

A large majority of students that live in the Local Market region are White, Hispanic/Latino or Asian.

In [ ]:
#Finds the various zip code counts of students that live in the Local Market region.
df_LocalMarket['ZIP'].value_counts().nlargest(10)

Most students in the Local Market region comes from the 95206 zip code.

In [ ]:
#Finds the count of the students HS GPA category that live in the Local Market region.
df_LocalMarket['HSGPA_cat'].value_counts()

About 47% of students in the Local Market had HS GPA of 3.60 or higher.

In [ ]:
#Finds male and female counts of students that live in the Northern California region.
df_NorthernCA['gender'].value_counts()

A large majority of students that live in the Northern CA region are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the Northern California region.
df_NorthernCA['ethnicity'].value_counts()

A large majority of students that live in the Northern CA region are White or Asian.

In [ ]:
#Finds the various zip code counts of students that live in the Northern California region.
df_NorthernCA['ZIP'].value_counts().nlargest(5)

A majority of the students who live in the Northern CA region come from the 94539 zip code.

In [ ]:
#Finds the count of the students HS GPA category that live in the Northern California region.
df_NorthernCA['HSGPA_cat'].value_counts()

About 35.2% of students in Northern CA had HS GPA of 3.60 or higher. About 16.2% of students had HS GPA of less than 3.10.

In [ ]:
#Finds male and female counts of students that live in the Southern California region.
df_SouthernCA['gender'].value_counts()

A large majority of students that live in the Southern CA region are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the Southern California region.
df_SouthernCA['ethnicity'].value_counts()

A large majority of students that live in the Southern CA region are White.

In [ ]:
#Finds the count of the students HS GPA category that live in the Southern California region.
df_SouthernCA['HSGPA_cat'].value_counts()

Only about 27% of Students in the Southern CA region had HS GPA over 27%. About 18% had HS GPA of less than 3.10.

In [ ]:
#Finds male and female counts of students that live in the International region.
df_International['gender'].value_counts()

A large majority of students that live in the International region are male.

In [ ]:
#Finds the various ethnicity counts of students that live in the International region.
df_International['ethnicity'].value_counts()

If a student lives in the International region, their ethnicity is also International.

In [ ]:
#Finds the count of the students HS GPA category that live in the International region.
df_International['HSGPA_cat'].value_counts()

About 27.5% of students in the International region had HS GPA of 3.60 or higher. About 23.2% of students had HS GPA less than 3.10.

In [ ]:
#Visualizes how many people graduated and did not by region of only males.
plt.figure(figsize=(12, 8)) 
sns.set_palette("pastel")
ax = sns.countplot(x = 'region',hue = 'graduated', data = df_male)
#plt.title("Male Graduation Rates by Region")
plt.legend(['Did Not Graduate', 'Graduated']);
for container in ax.containers:
    ax.bar_label(container)

Northern CA and Local Market had the highest college graduation rates among males.

In [ ]:
#Visualizes how many people graduated and did not by region of only females.
plt.figure(figsize=(12, 8)) 
sns.set_palette("pastel")
ax = sns.countplot(x = 'region',hue = 'graduated', data = df_female)
plt.title("Female Graduation Rates by Region", fontsize = 20)
plt.legend(['Did Not Graduate', 'Graduate'])
for container in ax.containers:
    ax.bar_label(container);

Students in the Northern CA and Local Market had the highest college graduation rates among Females as well. The amount of students who live in the regions of Western US and International are high but there sample size is lower.

Females had a higher college graduation rate than males in each region except females that live in the Southern California region.

In [ ]:
#Visualizes how many people graduated and did not by region of only students with a High School GPA under 3.10.
plt.figure(figsize=(12, 8)) 
sns.set_palette("pastel")
ax = sns.countplot(x = 'region',hue = 'graduated', data = df_HSGPA_less310)
plt.title("College Graduation by Region for Students with a High School GPA under 3.10", fontsize = 20)
plt.legend(['Did Not Graduate', 'Graduated'], loc='upper right')
for container in ax.containers:
    ax.bar_label(container);

Students with HS GPA less than 3.10 struggle in college. Students in the Local Market and Northern California regions have the highest population and their graduation rate is around 50%.

In [ ]:
#Visualizes how many people graduated and did not by region of only students with a High School GPA between 3.10 and 3.38.
plt.figure(figsize=(12, 8))
sns.set_palette("pastel")
ax = sns.countplot(x = 'region',hue = 'graduated', data = df_HSGPA_310_to338)
plt.title("College Graduation by Region for Students with a High School GPA between 3.10 and 3.38", fontsize = 20)
plt.legend(['Did Not Graduate', 'Graduated'])
for container in ax.containers:
    ax.bar_label(container);

The students whose HS GPA is between 3.10 and 3.38 have a higher college graduation rate than the students whose GPA is less than 3.10.

#### Slide 26 Ends

In [ ]:
#Visualizes how many people graduated and did not by ethnicity.
plt.figure(figsize=(16, 8))
sns.set_palette("pastel")
ax = sns.countplot(x = 'graduated',hue = 'ethnicity', data = df_combined)
plt.title("College Graduation by Ethnicity", fontsize = 20);
for container in ax.containers:
    ax.bar_label(container)

Every ethnicity category has more students who graduate from college vs did not graduate except for International. The majority of the dataset consists of White, Asian, and Hispanic/Latino.
White students have a college graduation rate of 74%, Asian students have a graduation rate of 66%, Hispanic/Latino students have a graduation rate of 64%.

#### Start of Slide 24

In [ ]:
#Visualizes how many people graduated and did not by HS GPA category.
plt.figure(figsize=(12, 8)) 
sns.set_palette("pastel")
ax = sns.countplot(x = 'HSGPA_cat',hue = 'graduated', data = df_combined)
plt.title("College Graduation by High School GPA Category", fontsize = 20)
plt.legend(['Did Not Graduate', 'Graduated']);
for container in ax.containers:
    ax.bar_label(container)

The higher a students HS GPA category, the more likely they are to graduate from college. Only 52% of students who have a HS GPA of less than 3.10 graduated from college.

In [ ]:
#Creates a dataframe of students I deem to have high HS GPA (Above 3.6).
df_high_HSGPA=df_combined_AE.loc[(df_combined_AE['HSGPA_cat'] == "3.60-3.82") | (df_combined_AE['HSGPA_cat'] == "3.83-4.00")]

#Creates a dataframe of students I deem to have low HS GPA (3.38 or less).
#I do not know the exact HS GPA of the students.
#The lowest category is students whose HS GPA is under 3.10.
df_low_HSGPA = df_combined_AE.loc[(df_combined_AE['HSGPA_cat'] == "Less 3.10") | (df_combined_AE['HSGPA_cat'] == "3.10-3.38")]


In [ ]:
#Finds the count of students with high HS GPA by region.
HighHSGPARegions = df_high_HSGPA['region'].value_counts().nlargest(5)
HighHSGPARegions

In [ ]:
#Finds the count of students with low HS GPA by region.
LowHSGPARegions = df_low_HSGPA['region'].value_counts().nlargest(5)
LowHSGPARegions

The smartest students came from the Local Market and Northern CA regions.

In [ ]:
#Plots a piechart of with high HS GPA by region.
labels = 'Local Market', 'Northern CA', 'Western US', 'Southern CA',  'International'
plt.pie(HighHSGPARegions, labels=labels, autopct='%1.1f%%')

In [ ]:
#Plots a piechart of with low HS GPA by region.
labels =  'Northern CA', 'Local Market', 'Southern CA', 'Western US',   'International'
plt.pie(LowHSGPARegions, labels=labels, autopct='%1.1f%%')

In [ ]:
#Finds the count of students with high HS GPA by major.
HighHSGPAMajors = df_high_HSGPA['major'].value_counts().nlargest(5)
HighHSGPAMajors

In [ ]:
#Finds the count of students with low HS GPA by major.
LowHSGPAMajors= df_low_HSGPA['major'].value_counts().nlargest(5)
LowHSGPAMajors

The proportion of students with high vs low HS GPA in each major is around 50% except for Bioengineering.
61% of the Bioengineering students had a high HS GPA.

In [ ]:
#Plots a piechart of with high HS GPA by major.
labels =  'Mechanical Engineering', 'Civil Engineering ', 'Bioengineering', 'Computer Science ',  'Exploratory (Engineering)'
plt.pie(HighHSGPAMajors, labels=labels, autopct='%1.1f%%')

In [ ]:
#Plots a piechart of with low HS GPA by major.
labels =  'Mechanical Engineering', 'Civil Engineering ', 'Computer Science ',  'Exploratory (Engineering)', 'Bioengineering',
plt.pie(LowHSGPAMajors, labels=labels, autopct='%1.1f%%')

In [ ]:
#Finds the count of students with high HS GPA by gender.
HighHSGPAGender = df_high_HSGPA['gender'].value_counts()
HighHSGPAGender

In [ ]:
#Finds the count of students with low HS GPA by gender.
LowHSGPAGender = df_low_HSGPA['gender'].value_counts()
LowHSGPAGender

There were more males with a low HS GPA than males with a high HS GPA.
There were more females with a high HS GPA than females with a low HS GPA.

In [ ]:
#Plots a piechart of with high HS GPA by gender.
labels =  "Male", "Female"

plt.pie(HighHSGPAGender, labels=labels, autopct='%1.1f%%')

In [ ]:
#Plots a piechart of with low HS GPA by gender.
labels =  "Male", "Female"
plt.pie(LowHSGPAGender, labels=labels, autopct='%1.1f%%')

In [ ]:
#Finds the count of students with high HS GPA by ethnicity.
HighHSGPAEthnicity = df_high_HSGPA['ethnicity'].value_counts().nlargest(5)
HighHSGPAEthnicity

In [ ]:
#Finds the count of students with low HS GPA by ethnicity.
LowHSGPAEthnicity = df_low_HSGPA['ethnicity'].value_counts().nlargest(5)
LowHSGPAEthnicity

White people had more than 50% of its students with a high HS GPA. 
All the other ethnicities had around 50% of its students with a high HS GPA.

In [ ]:
#Plots a piechart of with high HS GPA by ethnicity.
labels =  "White", "Asian", 'Hispanic/Latino', 'Two or more races', 'Race and ethnicity unknown'
plt.pie(HighHSGPAEthnicity, labels=labels, autopct='%1.1f%%')

In [ ]:
#Plots a piechart of with low HS GPA by ethnicity.
labels =  'White', 'Asian', 'Hispanic/Latino', 'International','Race and ethnicity unknown'
plt.pie(LowHSGPAEthnicity, labels=labels, autopct='%1.1f%%')

In [ ]:
#Finds the count of students with high HS GPA by zip code.
df_high_HSGPA['ZIP'].value_counts().nlargest(5)

In [ ]:
##inds the count of students with low HS GPA by zip code.
df_low_HSGPA['ZIP'].value_counts().nlargest(5)

The smartest students came from 95206 zip code. This zip code area covers the cities of Stockton, Taft, Mosswood, Gillis, and Holt.

#### End of Slide 24

#### Start of Slide 25

In [ ]:
#Plots a histogram of the mean cumulative college GPA of students.
# create 10 bins between 0 and 4 with equal width
bins = np.linspace(0, 4, 11)
fig, ax = plt.subplots(figsize=(10, 5))
# get the labels for the bins
bin_labels = [f"{bins[i]:.1f}-{bins[i+1]:.1f}" for i in range(len(bins)-1)]
# cut the GPA values into the bins
bin_counts = pd.cut(df_combined['Mean_GPA'], bins=bins, labels=bin_labels, include_lowest=True).value_counts(sort=False)
# create a bar graph of the bin counts with increased width
plt.bar(bin_counts.index.astype(str), bin_counts.values, width=0.8)
# set x-axis label
plt.xlabel('Cumulative University GPA')
# set y-axis label
plt.ylabel('Count')
#Set graph title
plt.title("Distribution of Cumulative GPA")
# show the plot
plt.show()

More than 100 people are below 2.0 cumulative college GPA. Find out what they have in common!

In [ ]:
#Creates a dataframe of students who mean cumulative college GPA is under 2.0.
under_2_Gpa = df_combined_AE.loc[df_combined_AE['Mean_GPA'] < 2.0]
under_2_Gpa.head()

In [ ]:
#Finds the number of rows (students) whose cumulative college GPA is under 2.0.

under_2_Gpa.shape[0]

In [ ]:
#Find the dataframe of students under 2.0 GPA who did not graduate.
under_2_Not_graduate = under_2_Gpa.loc[under_2_Gpa['graduated'] == 0]

under_2_Not_graduate.shape[0]


All the 183 students whose cumulative college was under 2.0 GPA did not graduate.

In [ ]:
# Finds out the percentage of students in each major under 2.0 cumulative college GPA against the whole population of students in the dataset.
under_2_Gpa['major'].value_counts()/df_combined['major'].value_counts()


For the students whose cumulative college GPA was under 2.0, the highest proportion against the total population of students in the dataset was students in the Engineering Physics major.

In [ ]:
# Gets the top 5 most frequent zip codes.
under_2_Gpa['ZIP'].value_counts().nlargest(5)

For the students whose cumulative college GPA was under 2.0, these are the top 5 zip codes.

In [ ]:
# Finds out the count of ethnicities for students whose cumulative college GPA is under 2.0.
under_2_Gpa['ethnicity'].value_counts()

For the students whose cumulative college GPA was under 2.0, the most common ethnicity was White. Find ratio!

In [ ]:
# Finds out the percentage of students in each ethnicity under 2.0 cumulative college GPA against the whole population of students in the dataset.
(under_2_Gpa['ethnicity'].value_counts())/(df_combined['ethnicity'].value_counts())

More analysis leads us to see that even though White students had the most number of students of under 2.0 cumulative college GPA, the proportion against the total population of White students was about 12.3%. The highest proportion was American Indian or Alaska Native at 20%.

##### Slide 25 ends

In [ ]:
#Visualizes how many people graduated from college and by number of unique majors.
plt.figure(figsize=(16, 8))
sns.set_palette("pastel")
ax = sns.countplot(x = 'graduated',hue = 'Num_of_Unique_Majors', data = df_combined)
plt.title("College Graduation by Number of Unique Majors", fontsize = 20);
for container in ax.containers:
    ax.bar_label(container)

A majority of the dataset consists of students who kept their original major throughout their studies. The graduation rate is 63%.

#### Start of Slide 23

In [ ]:
#Gpa's for majors with higher drop out rates

In [ ]:
#Creates a data frame of students who graduated from college.
df_graduated = df_combined_AE.loc[df_combined_AE['graduated'] == 1]
df_graduated.shape

In [ ]:
#Finds the dataframe of students who did not graduate from college.
df_Not_Graduate = df_combined.loc[df_combined['graduated'] == 0]
df_Not_Graduate.shape


In [ ]:
#Finds the dataframe of students who did not return for year 2 that did not graduate from college.
notReturn_2yr = df_Not_Graduate.loc[df_Not_Graduate['retrn_2yr'] == 0]
notReturn_2yr.shape

41% of the students who did not graduate did not return in the second year.

In [ ]:
#Finds the dataframe of students who did not return for year 3 that did not graduate from college.
notReturn_3yr = df_Not_Graduate.loc[df_Not_Graduate['retrn_3yr'] == 0]
notReturn_3yr.shape

Almost 70% of students who did not graduate did not return for their third year. They Dropped out after their 2nd year, and did not come back for their 3rd year.

In [ ]:
#Finds the dataframe of students who did not return for year 4 that did not graduate from college.
notReturn_4yr = df_Not_Graduate.loc[df_Not_Graduate['retrn_4yr'] == 0]
notReturn_4yr.shape

In [ ]:
#Finds the top 16 most common number of courses students took who did not graduate from college.
df_Not_Graduate['Num_of_Crses'].value_counts().nlargest(16)

For the people who did not graduate, the majority of the students only took less than 20 courses before they left.

In [ ]:
#Number of students who graduated in each major divided by the umber of students in the major

In [ ]:
graduation_rate_by_major = ((df_graduated['major'].value_counts())/(df_combined['major'].value_counts())).sort_values()

graduation_rate_by_major = round(graduation_rate_by_major, 3)
print(graduation_rate_by_major)


In [ ]:
#Number of students who graduated in each major

In [ ]:
df_graduated['major'].value_counts()

In [ ]:
#Number of students in each major

In [ ]:
df_combined['major'].value_counts().sort_values()

In [ ]:
#Finds the counts of major choices of students who did not graduate from college.
df_Not_Graduate['major'].value_counts().nlargest(10)

In [ ]:
#Find a dataframe of Mechanical Engineering students who did not graduate from college.
df_Not_Graduate_Mech_Eng = df_Not_Graduate.loc[df_combined["major"] == "Mechanical Engineering"]

##Find a dataframe of Civil Engineering students who did not graduate from college.
df_Not_Graduate_Civ_Eng = df_Not_Graduate.loc[df_combined["major"] == "Civil Engineering"]

##ind a dataframe of Computer Science students who did not graduate from college.
df_Not_Graduate_Comp_Sci = df_Not_Graduate.loc[df_combined["major"] == "Computer Science"]

In [ ]:
#Finds the number of rows (students) in the dataframe of Mechanical Engineering students who did not graduate.
df_Not_Graduate_Mech_Eng.shape[0]

In [ ]:
#Finds a dataframe of Mechanical Engineering Students under 3.0 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder3Gpa = df_Not_Graduate_Mech_Eng.loc[df_combined["Mean_GPA"] < 3.0]

In [ ]:
#Finds the number of rows (students) in the dataframe of Mechanical Engineering Students under 3.0 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder3Gpa.shape[0]

For the Mechanical Engineering major, 85% of the students who did not graduate had a mean gpa of under 3.0.

In [ ]:
#Finds a dataframe of Mechanical Engineering Students under 2.75 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder275Gpa = df_Not_Graduate_Mech_Eng.loc[df_combined["Mean_GPA"] < 2.75]

#Finds the number of rows (students) in the dataframe of Mechanical Engineering Students under 2.75 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder275Gpa.shape[0]

For the Mechanical Engineering, 82% of the students who did not graduate had a mean gpa of under 2.75.

In [ ]:
#Finds a dataframe of Mechanical Engineering Students under 2.75 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder225Gpa = df_Not_Graduate_Mech_Eng.loc[df_combined["Mean_GPA"] < 2.25]

#Finds the number of rows (students) in the dataframe of Mechanical Engineering Students under 2.25 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder225Gpa.shape[0]

For the Mechanical Engineering major, 63% of the students who did not graduate had a mean gpa of under 2.25.

In [ ]:
#Finds a dataframe of Mechanical Engineering Students under 2.0 cumulatitive college GPA.
df_Not_Graduate_Mech_Engunder2Gpa = df_Not_Graduate_Mech_Eng.loc[df_combined["Mean_GPA"] < 2]


#Finds the number of rows (students) in the dataframe of Mechanical Engineering Students under 2.0 cumulatitive college.
df_Not_Graduate_Mech_Engunder2Gpa.shape[0]

For the students Mechanical Engineering major, 48.5% of the students who did not graduate had a mean gpa of under 2.0.

#### End of Slide 23

In [ ]:
df_combined.retrn_3yr.value_counts()

In [ ]:
pd.crosstab(df_combined['retrn_3yr'],df_combined['grad_3yr'])

In [ ]:
print(df_combined.retrn_4yr.value_counts())
(pd.crosstab(df_combined['retrn_4yr'],df_combined['grad_4yr']))

In [ ]:
print(df_combined.retrn_5yr.value_counts())
pd.crosstab(df_combined['retrn_5yr'],df_combined['grad_5yr'])

In [ ]:
df_combined.retrn_5yr.value_counts()
pd.crosstab(df_combined['retrn_6yr'],df_combined['grad_6yr'])

In [ ]:
# for i in grad_data.columns:
#     plt.figure(figsize=(10,4))
#     sns.countplot(grad_data[i], palette = 'hls')
#     #sns.countplot(df_retrn_data[i], data = , palette = 'hls')
#     plt.show()

In [ ]:
#analyzing the dataset
advert_report = sv.analyze(df_combined)# importing sweetviz
#display the report
advert_report.show_html('grade.html')

In [ ]:
#checking amount of null values in the data
df_combined_short_Wo_Grad.info()

In [ ]:
#Droping PIDM as it is no longer need now that the data has been combined
df_combined = df_combined.drop('PIDM', axis=1)

### Machine Learning Modeling

### Preprocessing

In [ ]:
#Transform non-numeric columns into numerical columns
from sklearn.preprocessing import LabelEncoder

for column in df_combined.columns:
        if df_combined[column].dtype == np.number:
            continue
        df_combined[column] = LabelEncoder().fit_transform(df_combined[column])

In [ ]:
#Creating dataframes for students that did return the previous year

#use for return 3rd year
df_combined_retrn_2yr_1=df_combined[df_combined['retrn_2yr']==1]


#use for return 4th year
df_combined_retrn_3yr_1=df_combined_retrn_2yr_1[df_combined_retrn_2yr_1['retrn_3yr']==1]


#use for return 5th year
df_combined_retrn_4yr_1=df_combined_retrn_3yr_1[df_combined_retrn_3yr_1['retrn_4yr']==1]


#use for return 6th year
df_combined_retrn_5yr_1=df_combined_retrn_4yr_1[df_combined_retrn_4yr_1['retrn_5yr']==1]

#### Checking Absolute value of Correlations

In [ ]:
def order_corr(series):
    
    # Get the absolute values of the Series
    abs_series = series.abs()

    # Sort the Series by absolute value in descending order
    sorted_series = abs_series.sort_values(ascending=False)

    return sorted_series


In [ ]:
#retrn_2yr correlations

order_corr(df_combined.iloc[:, :-20].corr()['retrn_2yr'])

In [ ]:
#retrn_3yr correlations

order_corr(df_combined_retrn_2yr_1.iloc[:, :-15].corr()['retrn_3yr'])

In [ ]:
#retrn_4yr correlations

order_corr(df_combined_retrn_3yr_1.iloc[:, :-10].corr()['retrn_4yr'])

In [ ]:
#retrn_5yr correlations

order_corr(df_combined_retrn_4yr_1.iloc[:, :-5].corr()['retrn_5yr'])

In [ ]:
#retrn_6yr correlations

order_corr(df_combined_retrn_5yr_1.corr()['retrn_6yr'])

In [ ]:
# Function that runs the requested algorithm and returns the accuracy metrics
def fit_ml_algo(algo, X_train,y_train, cv):
    
    # One Pass
    model = algo.fit(X_train, y_train)
    acc = round(model.score(X_train, y_train) * 100, 2)
    
    # Cross Validation 
    train_pred = model_selection.cross_val_predict(algo,X_train,y_train,cv=cv,n_jobs = -1)
    
    # Cross-validation accuracy metric
    acc_cv = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)
    
    return train_pred, acc, acc_cv

In [ ]:
from sklearn import metrics

def lots_of_models(X_train,y_train):

    # Logistic Regression
    train_pred_log, acc_log, acc_cv_log = fit_ml_algo(LogisticRegression(random_state=123), X_train, y_train, 10)

    # Support Vector Machine
    train_pred_svc, acc_svc, acc_cv_svc = fit_ml_algo(SVC(random_state=123), X_train, y_train, 10)

    # K Nearest Neighbour
    train_pred_knn, acc_knn, acc_cv_knn = fit_ml_algo(KNeighborsClassifier(n_neighbors = 3), X_train, y_train, 10)

    # Gaussian Naive Bayes
    train_pred_gaussian, acc_gaussian, acc_cv_gaussian = fit_ml_algo(GaussianNB(), X_train, y_train, 10)

    # Decision Tree
    train_pred_dt, acc_dt, acc_cv_dt = fit_ml_algo(DecisionTreeClassifier(random_state=123), X_train, y_train, 10)

    # Random Forest
    train_pred_rf, acc_rf, acc_cv_rf = fit_ml_algo(RandomForestClassifier(n_estimators=100,random_state=123), X_train, y_train, 10)

    # Gradient Boosting Trees
    train_pred_gbt, acc_gbt, acc_cv_gbt = fit_ml_algo(GradientBoostingClassifier(random_state=123), X_train, y_train, 10)

    # Perceptron
    train_pred_perceptron, acc_perceptron, acc_cv_perceptron = fit_ml_algo(Perceptron(random_state=123), X_train, y_train, 10)

    models = pd.DataFrame({
        'Model': ['Logistic Regression','SVM','KNN','Naive Bayes','Decision Tree','Random Forest','Gradient Boosting','Perceptron'],
        'Acc_CV': [acc_cv_log, acc_cv_svc, acc_cv_knn, acc_cv_gaussian, acc_cv_dt, acc_cv_rf, acc_cv_gbt, acc_cv_perceptron]})
    
    models_sorted = models.sort_values(by='Acc_CV', ascending=False)

    return models_sorted


In [ ]:
#Random Forest Grid Search 

def grid_search_rf(rf_params,X_train,y_train): 

    # Create a dictionary to store the best parameters
    best_params = {}

    # Perform grid search cross-validation for Random Forest

    rf = RandomForestClassifier(random_state=123)
    grid_search_rf = GridSearchCV(rf, rf_params, cv=5,n_jobs=7)
    grid_search_rf.fit(X_train, y_train)
    best_params['Random Forest'] = grid_search_rf.best_params_

    # Random Forest
    train_pred_rf, acc_rf, acc_cv_rf= fit_ml_algo(RandomForestClassifier(**best_params['Random Forest'],random_state=123),X_train, y_train,10)

    models = pd.DataFrame({
        'Model': ['Random Forest'],
        'Acc_CV': [acc_cv_rf]
    })

    return models, best_params

In [ ]:
#Gradiant Boost Grid Search 

def grid_search_gbt(rf_params,X_train,y_train): 

    # Create a dictionary to store the best parameters for each model
    best_params = {}
    
    # Perform grid search cross-validation for Gradient Boost
    gb = GradientBoostingClassifier(random_state=123)
    grid_search_gb = GridSearchCV(gb, gb_params, cv=5,n_jobs=7)
    grid_search_gb.fit(X_train, y_train)
    best_params['Gradient Boosting'] = grid_search_gb.best_params_

    # Gradient Boosting
    train_pred_gbt, acc_gbt, acc_cv_gbt = fit_ml_algo(GradientBoostingClassifier(**best_params['Gradient Boosting'],random_state=123),X_train, y_train,10)


    models = pd.DataFrame({
        'Model': ['Gradient Boosting'],
        'Acc_CV': [acc_cv_gbt]
    })
    return models, best_params

In [ ]:
#SVM Grid Search 

def grid_search_svm(svm_params, X_train, y_train):

    # Create a dictionary to store the best parameters for each model
    best_params = {}
    
    # Perform grid search cross-validation for SVM
    svm = SVC(random_state=123)
    grid_search_svm = GridSearchCV(svm, svm_params, cv=5, n_jobs=7)
    grid_search_svm.fit(X_train, y_train)
    best_params['Support Vector Machine'] = grid_search_svm.best_params_
    
    # SVM
    train_pred_svm, acc_svm, acc_cv_svm = fit_ml_algo(SVC(**best_params['Support Vector Machine'],random_state=123),X_train, y_train,10)


    models = pd.DataFrame({
        'Model': ['Support Vector Machine'],
        'Acc_CV': [acc_cv_svm]
    })
    return models, best_params

In [ ]:
### KNN Grid Search 

def grid_search_knn(knn_params):
    
    # Create a dictionary to store the best parameters for each model
    best_params = {}
    
    # Perform grid search cross-validation for KNN
    knn = KNeighborsClassifier()
    grid_search_knn = GridSearchCV(knn, knn_params, cv=5, n_jobs=7)
    grid_search_knn.fit(X_train, y_train)
    best_params['K Nearest Neighbour'] = grid_search_knn.best_params_

    # K-Nearest Neighbors
    train_pred_knn, acc_knn, acc_cv_knn = fit_ml_algo(KNeighborsClassifier(**best_params['K Nearest Neighbour']), X_train, y_train, 10)

    models = pd.DataFrame({
        'Model': ['K-Nearest Neighbors'],
        'Acc_CV': [acc_cv_knn]
    })
    return models, best_params


In [ ]:
def grid_search_lr(lr_params):

    # Create an instance of LogisticRegression
    logreg = LogisticRegression(random_state=123)

    # Perform grid search cross-validation for logistic regression
    grid_search_logreg = GridSearchCV(logreg, lr_params, cv=5, n_jobs=7)
    grid_search_logreg.fit(X_train, y_train)

    # Extract the best parameters and fit the logistic regression model with the best parameters
    best_params['Logistic Regression'] = grid_search_logreg.best_params_

    
    # Logistic Regression
    train_pred_log, acc_log, acc_cv_log = fit_ml_algo(LogisticRegression(**best_params['Logistic Regression'],random_state=123), X_train, y_train, 10)


    models = pd.DataFrame({
        'Model': ['Logistic Regression'],
        'Acc_CV': [acc_cv_log]
    })

    return models, best_params

In [ ]:
def feature_importance_rf(best_params):
    # Create a RandomForestClassifier with the specified hyperparameters
    
    rf = RandomForestClassifier(**best_params['Random Forest'],random_state=123)

    # Fit the RandomForestClassifier to your data
    rf.fit(X_train, y_train)  # Replace X_train and y_train with your actual training data

    # Get feature importances as a numpy array
    importances = rf.feature_importances_

    # Create a dataframe to store the feature importances
    df_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

    # Sort the dataframe by feature importance in descending order
    df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)
    
    return df_feature_importance

In [ ]:
def feature_importance_gbt(best_params):
    
    # Create a GradientBoostingClassifier with the specified hyperparameters
    gbt = GradientBoostingClassifier(**best_params['Gradient Boosting'],random_state=123)

    # Fit the RandomForestClassifier to your data
    gbt.fit(X_train, y_train)  # Replace X_train and y_train with your actual training data

    # Get feature importances as a numpy array
    importances = gbt.feature_importances_

    # Create a dataframe to store the feature importances
    df_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

    # Sort the dataframe by feature importance in descending order
    df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)
    
    return df_feature_importance

In [ ]:
def feature_importance_svm(best_params):
    # Create a Support Vector Machine (SVM) classifier with the specified hyperparameters
    svm = SVC(**best_params['Support Vector Machine'],random_state=123)

    # Fit the SVM classifier to your data
    svm.fit(X_train, y_train)

    # Calculate permutation importances
    result = permutation_importance(svm, X_train, y_train, n_repeats=10, random_state=0)
    importances = result.importances_mean

    # Create a dataframe to store the feature importances
    df_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

    # Sort the dataframe by feature importance in descending order
    df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)

    return df_feature_importance


In [ ]:
def feature_importance_knn(best_params):
    # Create a KNN classifier with the specified hyperparameters
    knn = KNeighborsClassifier(**best_params['K Nearest Neighbour'])

    # Fit the KNN classifier to your data
    knn.fit(X_train, y_train)

    # Calculate permutation importances
    result = permutation_importance(knn, X_train, y_train, n_repeats=10, random_state=0)
    importances = result.importances_mean

    # Create a dataframe to store the feature importances
    df_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

    # Sort the dataframe by feature importance in descending order
    df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)

    return df_feature_importance

In [ ]:
def feature_importance_lr(best_params):

    # Create an instance of LogisticRegression with the best hyperparameters
    logreg = LogisticRegression(**best_params['Logistic Regression'],random_state=123)

    # Fit the logistic regression model to the training data
    logreg.fit(X_train, y_train)

    # Get coefficients as feature importances
    importances = logreg.coef_[0]

    # Create a dataframe to store the feature importances
    df_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})

    # Sort the dataframe by feature importance in descending order
    df_feature_importance = df_feature_importance.sort_values(by='Importance', ascending=False)

    return df_feature_importance

### Predictive Models for 2nd Year Student Return

#### 2nd Year Intitial Model

In [ ]:
#train data split

#X value
X = df_combined.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=X.drop(['median_household_income'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-20]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

This random forest model that predicts 2nd-year returns is very accurate with an accuracy of 97.78%. However, the imbalance of feature importance, with Num_of_Crses being overwhelmingly greater than the other features, could indicate that some of the other features should be removed to create an improved model.

#### 2nd Year Model using Top Feature Importance Values

In [ ]:
#train data split

selected_features = df_feature_importance.head(8)['Feature'].values

#X value
X = df_combined.loc[:, selected_features]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

This model had the same accuracy as 97.78%. This meant that there was no notable increase in the accuracy even with the removal of potential noise from low feature importance columns. Interestingly the model seems to put even greater emphasis on Num_of_Crses, which could indicate that an even simpler model may be all that is required to create an accurate model on 2nd-year return students.

#### 2nd Year Model Excluding Num_of_Crses from X Values

In [ ]:
#train data split

#X value
X = df_combined.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=X.drop(['median_household_income','Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-20]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#checking feature importance of first model 

gb_params = {
    'n_estimators': [50,75,100, 200, 300], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7], 
    'min_samples_split': [2, 4, 8], 
    'min_samples_leaf': [1, 2, 4] 
}

models, best_params = grid_search_gbt(gb_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_gbt(best_params)
df_feature_importance

This model reconfirms my suspicion that the feature Num_of_Crses plays a vital role in creating an optimal model since the accuracy in this model, 89.63%, is significantly less than the previous models that did use Num_of_Crses. This model does interestingly show that it places great importance on the Mean_GPA feature, so the importance of this feature will be checked upon in the next model that removes this feature as well.

####  2nd Year Model Excluding Num_of_Crses and Mean_GPA from X Values

In [ ]:
#train data split

#X value
X = df_combined.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=X.drop(['Mean_GPA','Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-20]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#checking feature importance of first model 

gb_params = {
    'n_estimators': [50,75,100, 200, 300], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7], 
    'min_samples_split': [2, 4, 8], 
    'min_samples_leaf': [1, 2, 4] 
}

models, best_params = grid_search_gbt(gb_params,X_train,y_train)

display(models,best_params)

#feature importance
df_feature_importance=feature_importance_gbt(best_params)
df_feature_importance

Again it is apparent that the model is not performing as well as the models that did include Num_of_Crses. This again reconfirms that Num_of_Crses plays a notable role in the creation of an optimal model to predict students returning during their 2nd year. Another interesting finding is despite Mean_GPA seemingly like another very important feature in previous models, the removal of the feature did not greatly decrease this model's accuracy relative to the previous model. 

#### 2nd Year Model with PCA

In [ ]:
#train data split

#X value
X = df_combined.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis=1)

# Dropping classes that Students haven't taken yet
X = X.iloc[:, :-20]

#PCA
columns_for_pca = X.columns.tolist()
columns_for_pca.remove('Num_of_Crses')
columns_for_pca.remove('Mean_GPA')

# Perform PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X[columns_for_pca])

# Create new columns in X for the PCA results
for i in range(4):
    X[f'PCA_Component_{i+1}'] = X_pca[:,i]

# Drop the original columns used for PCA
X.drop(columns=columns_for_pca, inplace=True, axis=1)

#Y value
Y_2yr = df_combined['retrn_2yr']

#Split
X_train, X_test, y_train, y_test = train_test_split(X, Y_2yr, test_size=0.20, random_state=123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance
df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

In [ ]:
#tunning hyperparameters in gradiant boost

gb_params = {
    'n_estimators': [50,75,100, 200, 300], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7], 
    'min_samples_split': [2, 4, 8], 
    'min_samples_leaf': [1, 2, 4] 
}

models, best_params = grid_search_gbt(gb_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_gbt(best_params)
df_feature_importance

To reduce the noise from the lower-importance features in the data, PCA was used. The model that used PCA had an accuracy of 97.61% which meant it did a lot better than the last two models. However, it did do a bit worse than the first two models. This makes it very apparent that Num_of_Crses may be by far the single most important feature in predicting 2nd-year student return. 

#### 2nd Year Model Using Only Num_of_Crses and Mean_GPA for X Values

In [ ]:
#train data split

#X value
X = df_combined[['Num_of_Crses', 'Mean_GPA']]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in SVM

svm_params = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf','linear']}

models, best_params = grid_search_svm(svm_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_svm(best_params)
df_feature_importance

Since Num_of_Crses and Mean_GPA seemed to be the most important features in model performance, this model used only those two features. This model performed better than all other models with an accuracy of 97.87%. Since this model is also the simplest model and easiest to explain, this model will be chosen to be our final model for our model to predict 2nd year return.

#### Saving the Best Performing Model for 2nd Year Return

In [ ]:
#train data split

#X value
X = df_combined[['Num_of_Crses', 'Mean_GPA']]

#Y value
Y_2yr=df_combined['retrn_2yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_2yr, test_size=0.20, random_state = 123)

#Inputting hyper-parameters
svc_model = SVC(**{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'},random_state=123)

#recreating best model
svc_model.fit(X_train, y_train)

#confusion matrix
y_pred = svc_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)


joblib.dump(svc_model, '2nd_year_return_model.pkl')

### Predictive Models for 3rd Year Student Return

#### 3rd Year Intitial Model

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-15]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The random forest model that predicts 3rd-year returns is very accurate with an accuracy of 96.53%. However, the imbalance of feature importance, with Num_of_Crses being overwhelmingly greater than the other features, could indicate that some of the other features should be removed to create an improved model.

#### 3rd year  Model using Top Feature Importance Values

In [ ]:
#train data split

selected_features = df_feature_importance.head(14)['Feature'].values

#X value
X = df_combined_retrn_2yr_1.loc[:, selected_features]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

This  model achieved an accuracy of 96.42%, which is slightly lower than expected. However, even after limiting the analysis to the top feature importance values, the difference in accuracy was not significantly different between this model and the intial model. This model further emphasizes the critical role of the Num_of_Crses feature, which had a feature importance score that was over 40 times higher than the next most important feature. To confirm the crucial nature of this feature, the next model will exclude "Num_of_Crses" to assess its impact on performance.

#### 3rd year model with no Num_of_Crses in X values

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=X.drop(['Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-15]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

With an accuracy of 92.34%, this model performed significantly worse than the previous two models. Interestingly, the feature importance analysis revealed that the Mean_GPA feature had a substantial impact on the model's performance, suggesting its potential importance in creating an optimized model. To further investigate this feature's significance, the next model will exclude Mean_GPA to assess the impact of its removal on the model's performance.

#### 3rd year model with no Num_of_Crses, Mean_GPA in X values

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=X.drop(['Mean_GPA','Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-15]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The accuracy of this model was 91.01%, which indicates a slight decrease in performance compared to the previous model. The drop in accuracy is approximately 1%. This observation suggests that Mean_GPA might still be a valuable feature to include in future models to optimize their performance.

#### 3rd year model with PCA

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis=1)

# Dropping classes that Students haven't taken yet
X = X.iloc[:, :-15]

#PCA
columns_for_pca = X.columns.tolist()
columns_for_pca.remove('Num_of_Crses')
columns_for_pca.remove('Mean_GPA')

# Perform PCA
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X[columns_for_pca])

# Create new columns in X for the PCA results
for i in range(4):
    X[f'PCA_Component_{i+1}'] = X_pca[:,i]

# Drop the original columns used for PCA
X.drop(columns=columns_for_pca, inplace=True, axis=1)

#Y value
Y_3yr = df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train, X_test, y_train, y_test = train_test_split(X, Y_3yr, test_size=0.20, random_state=123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance of first model 

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

This model achieved an accuracy of 96.63%, which is the highest accuracy obtained thus far. Although the improvement over the first two models is not significant, it is still a noteworthy advancement considering the relatively small increase of only 0.1% in accuracy. Additionally, the feature importance analysis indicates that the most valuable features are Num_of_Crses and Mean_GPA, which have much higher importance scores compared to the other features. This finding will be further investigated in the next model, which will exclusively use Num_of_Crses and Mean_GPA to develop the model.

#### 3rd year Model with only Num_of_Crses and Mean_GPA

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=df_combined_retrn_2yr_1[['Mean_GPA','Num_of_Crses']]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in SVM

svm_params = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf','linear']}

models, best_params = grid_search_svm(svm_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_svm(best_params)
df_feature_importance

Since Num_of_Crses and Mean_GPA seemed to be the most important features in model performance, this model used only those two features. The model achieved an accuracy of 96.73%, surpassing the performance of all other models. Furthermore, this model is the simplest and easiest to explain, making it an ideal choice as the final model to predict 3rd-year return.

#### Saving the Best Performing Model for 3rd Year Return

In [ ]:
#train data split

#X value
X = df_combined_retrn_2yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Feature Selection from correlation
X=df_combined_retrn_2yr_1[['Mean_GPA','Num_of_Crses']]

#Y value
Y_3yr=df_combined_retrn_2yr_1['retrn_3yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_3yr, test_size=0.20, random_state = 123)

#Inputting hyper-parameters
svc_model = SVC(**{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'},random_state=123)

#recreating model
svc_model.fit(X_train, y_train)

#confusion matrix
y_pred = svc_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)

joblib.dump(svc_model, '3rd_year_return_model.pkl')

### Predictive Models for 4th Year Student Return

#### 4th Year Intitial Model

In [ ]:
#train data split

#X value
X = df_combined_retrn_3yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-10]

#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance 

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The random forest model that predicts 2nd-year returns is very accurate with an accuracy of 96.24%. However, the imbalance of feature importance, with Num_of_Crses being overwhelmingly greater than the other features, indicating that the removal of other features could aid in creating a more optimal model. So a filter of the highest feature importances will be used in the next model.

#### 4th year return Model with feature importance

In [ ]:
#train data split

selected_features = df_feature_importance.head(8)['Feature'].values

#X value
X = df_combined_retrn_3yr_1.loc[:, selected_features]

#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

Despite the removal of a large set of features, this model had the same accuracy as the previous model. This model had an accuracy of 96.24%. The model also had two features that stood out as having significant higher values in their feature importance than other features which were the features Num_of_Crses and Mean_GPA. In order to further check on the impact of these two features the Num_of_Crses will be removed to see how a model trained without that feature would perform.

#### 4th year model with no Num_of_Crses in X values

In [ ]:
#train data split

#X value
X = df_combined_retrn_3yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Testing no Num_of_Crses
X=X.drop(['Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-10]

#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance 

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

This trained model had a accuracy of 93.61%. As with the previous year models, when removing the Num_of_Crses, the model has a worse performance than models that do have this feature. This further reinforces the idea that Num_of_Crses plays a critical role in creating an optimal model in predicting student reuturn.

#### 4th year model with no Num_of_Crses, Mean_GPA in X values

In [ ]:
#train data split

#X value
X = df_combined_retrn_3yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Testing no Num_of_Crses
X=X.drop(['Mean_GPA','Num_of_Crses'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-10]

#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#checking feature importance of RF
rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#checking feature importance 

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The accuracy of this model was 93.84%, which is the lowest among all the 4th year return models. It is evident yet again that the model's performance is inferior to the models that included both Num_of_Crses and Mean_GPA. This finding reinforces the notion that Num_of_Crses plays a significant role in creating an optimal model to predict students returning during their 4th year, and although not as critical, Mean_GPA also plays a notable role in improving the model's performance.

#### 4th year Model with PCA

In [ ]:
#train data split

#X value
X = df_combined_retrn_3yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

# Dropping classes that Students haven't taken yet
X = X.iloc[:, :-10]

#PCA
columns_for_pca = X.columns.tolist()
columns_for_pca.remove('Num_of_Crses')
columns_for_pca.remove('Mean_GPA')

# Perform PCA
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X[columns_for_pca])

# Create new columns in X for the PCA results
for i in range(5):
    X[f'PCA_Component_{i+1}'] = X_pca[:,i]

# Drop the original columns used for PCA
X.drop(columns=columns_for_pca, inplace=True, axis=1)

#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train, X_test, y_train, y_test = train_test_split(X, Y_4yr, test_size=0.20, random_state=123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance
df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

In [ ]:
#tunning hyperparameters in gradiant boost

gb_params = {
    'n_estimators': [75,100,200], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7], 
    'min_samples_split': [2, 4, 8], 
    'min_samples_leaf': [1, 2, 4] 
}

models, best_params = grid_search_gbt(gb_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_gbt(best_params)
df_feature_importance

When applying PCA to reduce noise in the model, the resulting accuracy was 95.32%, slightly lower than that of the original two models. The feature importance analysis of this model reaffirmed that Num_of_Crses remains a crucial determinant.

#### 4th year Model with only Num_of_Crses and Mean_GPA

In [ ]:
#train data split

#X value
X=df_combined_retrn_3yr_1[['Mean_GPA','Num_of_Crses']]


#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in KNN

knn_params = { 'n_neighbors' : [2,3,5,7,9],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

models, best_params = grid_search_knn(knn_params)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_knn(best_params)
df_feature_importance

By utilizing only the two features, Num_of_Crses and Mean_GPA, to create a predictive model, I achieved an accuracy of 96.12%. This is only marginally lower than the first two models, with a difference of just 0.1%. Despite its slightly lower accuracy, I will opt to use this model, applying the principle of Occam's razor, as a K Nearest Neighbour model is simpler than the Random Forest models.

#### Saving the Best Performing Model for 4th Year Return

In [ ]:
#train data split

#X value
X=df_combined_retrn_3yr_1[['Mean_GPA','Num_of_Crses']]


#Y value
Y_4yr=df_combined_retrn_3yr_1['retrn_4yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_4yr, test_size=0.20, random_state = 123)

#Inputting hyper-parameters
knn_model = KNeighborsClassifier(**{'metric': 'minkowski','n_neighbors': 7,'weights': 'uniform'})

knn_model.fit(X_train, y_train)

#confusion matrix
y_pred = knn_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)

joblib.dump(knn_model, '4th_year_return_model.pkl')

### Predictive Models for 2nd Year Student Return

#### 5th Year Intitial Model

In [ ]:
#train data split

#X value
X = df_combined_retrn_4yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-5]

#Y value
Y_5yr=df_combined_retrn_4yr_1['retrn_5yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_5yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The intial model for predicting 5th year return has the worst performance out of all the previous models when predicting their return with a 80.17% accuracy. All the previous models had an accuracy of 90% and higher. This is a slight concern as this could indicate issues with the data used for modeling for 5th year return or it could just mean that the features are not a great predictor on student reutrn. Inorder to further investigate this issue a filter of the top feature importance features will be used to make a model.

#### 5th Year Model using Top Feature Importance Values

In [ ]:
#train data split

selected_features = df_feature_importance.head(26)['Feature'].values

#X value
X = df_combined_retrn_4yr_1.loc[:, selected_features]

#Y value
Y_5yr=df_combined_retrn_4yr_1['retrn_5yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_5yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

By taking only the columns with higher feature importance, this model had a better accuracy with an accuracy of 81.88%. Though the increase was only around 1%, it still increased which could indicate that there may be too much noise in the model to accuractly predict the return of 5th year students. The next model will use PCA to try to tackle this issue.

#### 5th year Model with PCA

In [ ]:
#train data split

#X value
X = df_combined_retrn_4yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis=1)

# Dropping classes that Students haven't taken yet
X = X.iloc[:, :-5]

#PCA
columns_for_pca = X.columns.tolist()
columns_for_pca.remove('Num_of_Crses')
columns_for_pca.remove('Mean_GPA')

# Perform PCA
pca = PCA(n_components=6)
X_pca = pca.fit_transform(X[columns_for_pca])

# Create a new column in X for the PCA result
for i in range(6):
    X[f'PCA_Component_{i+1}'] = X_pca[:, i]

# Drop the original columns used for PCA
X.drop(columns=columns_for_pca, inplace=True)

# Y value
Y_5yr = df_combined_retrn_4yr_1['retrn_5yr']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, Y_5yr, test_size=0.20, random_state=123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in gradiant boost

gb_params = {
    'n_estimators': [75,100,200], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7], 
    'min_samples_split': [2, 4, 8], 
    'min_samples_leaf': [1, 2, 4] 
}

models, best_params = grid_search_gbt(gb_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_gbt(best_params)
df_feature_importance

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

Contrary to expectation, the models created with PCA failed to achieve an accuracy of 80%. These models performed worse than the previous two, suggesting that there may be too much noise in the data, which even PCA was unable to remove. To improve the model's performance in predicting 5th year students, the next model will be using only Num_of_Crses and Mean_GPA as features.

#### 5th Year Model Using Only Num_of_Crses and Mean_GPA for X Values 

In [ ]:
#train data split

#X value
X = df_combined_retrn_4yr_1[['Num_of_Crses', 'Mean_GPA']]

#Y value
Y_5yr=df_combined_retrn_4yr_1['retrn_5yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_5yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in Logistic Regression
lr_params = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

models, best_params = grid_search_lr(lr_params)
display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_lr(best_params)
df_feature_importance

In [ ]:
#tunning hyperparameters in SVM

svm_params = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf','linear']}

models, best_params = grid_search_svm(svm_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_svm(best_params)
df_feature_importance

For the first time in all the different return year models, the models created with only Num_of_Crses and Mean_GPA as input features did not perform well. None of the models were able to achieve an accuracy of 80%, and all were outperformed by the previous models. The current method of subsetting the data may be throwing off the model, as it keeps all students who returned in the previous year. Given that most students graduate in four years, this could be adding a lot of noise and leading to subpar predictive models for 5th year return. I will save the model that utilized the features with the highest importance, as it performed the best.

#### Saving the Best Performing Model for 5th Year Return

In [ ]:
#train data split

#X value
X = df_combined_retrn_4yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

# Dropping classes that Students haven't taken yet
X=X.iloc[:, :-5]

#Y value
Y_5yr=df_combined_retrn_4yr_1['retrn_5yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_5yr, test_size=0.20, random_state = 123)


#tunning hyperparameters in random forest model

rf_params = {'bootstrap': [True],
             'criterion': ['entropy'],
             'max_depth': [None],
             'max_features': [None],
             'min_samples_leaf': [2],
             'min_samples_split': [8],
             'n_estimators': [50]}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

#Feature importance

df_feature_importance=feature_importance_rf(best_params)


#train data split

selected_features = df_feature_importance.head(26)['Feature'].values

#X value
X = df_combined_retrn_4yr_1.loc[:, selected_features]

#Y value
Y_5yr=df_combined_retrn_4yr_1['retrn_5yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_5yr, test_size=0.20, random_state = 123)

#Inputting hyper-parameters
rf_model = RandomForestClassifier(**{'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50},random_state=123)

#train model
rf_model.fit(X_train, y_train)

#creating confusion matrix
y_pred = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)

#saving model
joblib.dump(rf_model, '5th_year_return_model.pkl')


### Predictive Models for 6th Year Student Return

#### Initial Model

In [ ]:
#train data split

#X value
X = df_combined_retrn_5yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Y value
Y_6yr=df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_6yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

# feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The initial model for predicting 6th year return achieved an accuracy of 87.76%. Compared to all the other models, this model had the most balanced feature importance values, but it still followed the pattern seen in other models where Num_of_Crses and Mean_GPA had the highest importance values. Given the large number of features in the model, I will filter out the features with low importance and keep only the ones with the highest importance.

####  6th Year Model using Top Feature Importance Values

In [ ]:
#train data split

selected_features = df_feature_importance.head(8)['Feature'].values

#X value
X = df_combined_retrn_5yr_1.loc[:, selected_features]

#Y value
Y_6yr=df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_6yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

The model showed a slight improvement, with an accuracy of 89.12%, which is about 2% higher than the previous model. This suggests that the feature filtering process did improve the model performance, by removing noise caused by less important features. Additionaly the feature importance of the features to more balanced. To further improve the model by reducing noise, I will apply PCA in the next model.

#### 6th year model with PCA

In [ ]:
#train data split

#X value
X = df_combined_retrn_5yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis=1)


#PCA
columns_for_pca = X.columns.tolist()
columns_for_pca.remove('Num_of_Crses')
columns_for_pca.remove('Mean_GPA')

# Perform PCA
pca = PCA(n_components=7)
X_pca = pca.fit_transform(X[columns_for_pca])

# Create a new column in X for the PCA result
for i in range(7):
    X[f'PCA_Component_{i+1}'] = X_pca[:, i]

# Drop the original columns used for PCA
X.drop(columns=columns_for_pca, inplace=True)

#Y value
Y_6yr = df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train, X_test, y_train, y_test = train_test_split(X, Y_6yr, test_size=0.20, random_state=123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")


In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in random forest model

rf_params = {
    'max_depth': [None,2,3,4],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2',None],
    'criterion':['gini','entropy'],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'n_estimators': [1,2,4,50,75]
}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

display(models,best_params)

#Feature importance
df_feature_importance=feature_importance_rf(best_params)
df_feature_importance

Using PCA the model again out performed the intial model by about 1%. However this model did not beat the previous model that used the higest feature importance features. 

#### 6th Year Model Using Only Num_of_Crses and Mean_GPA for X Values 

In [ ]:
#train data split

#X value
X = df_combined_retrn_5yr_1[['Num_of_Crses', 'Mean_GPA']]

#Y value
Y_6yr=df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_6yr, test_size=0.20, random_state = 123)

print(f"Train data shape: {X_train.shape}, Test Data Shape {X_test.shape}")

In [ ]:
#Model accuracy comparison

models_sorted=lots_of_models(X_train,y_train)

display(models_sorted)

print(models_sorted.iloc[0][0],"is the best model")

In [ ]:
#tunning hyperparameters in SVM

svm_params = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf','linear']}

models, best_params = grid_search_svm(svm_params,X_train,y_train)

display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_svm(best_params)
df_feature_importance

In [ ]:
#tunning hyperparameters in Logistic Regression
lr_params = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

models, best_params = grid_search_lr(lr_params)
display(models,best_params)

#Feature importance

df_feature_importance=feature_importance_lr(best_params)
df_feature_importance

Once again, the models that used only Num_of_Crses and Mean_GPA as features performed better than the initial model. However, none of these models outperformed the model that used the top feature importance values. Therefore, I will save the model that performed the best, which is the one that used the top feature importance values, as our final model for predicting 6th year return.

#### Saving the Best Performing Model for 6th Year Return

In [ ]:
#train data split

#X value
X = df_combined_retrn_5yr_1.drop(['retrn_2yr','retrn_3yr','retrn_4yr','retrn_5yr','retrn_6yr','grad_3yr','grad_4yr','grad_5yr','grad_6yr','graduated','term_code_key'], axis =1)

#Y value
Y_6yr=df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_6yr, test_size=0.20, random_state = 123)

#tunning hyperparameters in random forest model

rf_params = {'bootstrap': [False],
  'criterion': ['gini'],
  'max_depth': [None],
  'max_features': ['sqrt'],
  'min_samples_leaf': [2],
  'min_samples_split': [8],
  'n_estimators': [4]}

models, best_params = grid_search_rf(rf_params,X_train,y_train)


# feature importance

df_feature_importance=feature_importance_rf(best_params)
df_feature_importance


#train data split

selected_features = df_feature_importance.head(8)['Feature'].values

#X value
X = df_combined_retrn_5yr_1.loc[:, selected_features]

#Y value
Y_6yr=df_combined_retrn_5yr_1['retrn_6yr']

#Split
X_train,X_test, y_train, y_test = train_test_split(X,Y_6yr, test_size=0.20, random_state = 123)

#tunning hyperparameters in random forest model

rf_params = {'bootstrap': [True],
  'criterion': ['gini'],
  'max_depth': [4],
  'max_features': ['log2'],
  'min_samples_leaf': [1],
  'min_samples_split': [4],
  'n_estimators': [75]}

models, best_params = grid_search_rf(rf_params,X_train,y_train)

#Feature importance
df_feature_importance=feature_importance_rf(best_params)


#Inputting hyper-parameters
rf_model = RandomForestClassifier(**best_params['Random Forest'],random_state=123)

#Recreating model
rf_model.fit(X_train, y_train)

#creating confusion matrix
y_pred = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)

joblib.dump(rf_model, '6th_year_return_model.pkl')